In [2]:
# This notebook is used to randomly define the weights given to events based on the different uncertainty
# components we take into account. The notebook produces a set of .txt files which can then be read by the
# script that runs on the grid in order to produce different toys that reflect the uncertainty estimation.

In [ ]:
from bisect import bisect as bisect
import numpy as np
from csv import reader
import numpy.random as rnd
import ROOT
import os
import time

%jsroot on

In [ ]:
def create_toy_ratio(PCA_limit=100):
    Nbins = np.genfromtxt('Nbins.csv', delimiter=',')[1:-2]
    Nbins_sum = [Nbins[:i].sum() for i in range(len(Nbins))]
    centers = []
    with open('BinCenters.csv', 'r') as read_obj:
        csv_reader = reader(read_obj)
        for row in csv_reader:
            centers.append(np.array(row).astype(float)[1:])
    centers = centers[1:-1]
    ## get uncertainties (PCs)
    uncertainties = []
    for k in range(PCA_limit):
        file = ROOT.TFile.Open("param_pca_"+str(k)+"TH1.root")
        hist = file.Get("ND_nu_numu_1D")
        total = []
        for i in range(len(Nbins)):
            total.append([hist.GetBinContent(int(Nbins_sum[i]+j)) for j in range(int(Nbins[i]))])
        uncertainties.append(total)
    ## get beamline systematics
    directory = os.getcwd()
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith("0TH1.root"):
            break
        elif filename.endswith("TH1.root") or filename.endswith(".py"): 
            file = ROOT.TFile.Open(filename)
            hist = file.Get("ND_nu_numu_1D")
            total = []
            for i in range(len(Nbins)):
                total.append([hist.GetBinContent(int(Nbins_sum[i]+j)) for j in range(int(Nbins[i]))])
            uncertainties.append(total)
            continue
        else:
            continue
    
    total_dev = np.zeros(846) # total number of bins
    for l in range(len(uncertainties)):
        variation = rnd.normal()
        uncertainties_1D = [uncertainties[l][i][j] for i in range(len(centers)) for j in range(len(centers[i]))]
        uncertainties_varied = np.array([uncertainty_val*variation for uncertainty_val in uncertainties_1D])
        total_dev += uncertainties_varied
    total_dev_by_oabin = [np.array([total_dev[int(Nbins_sum[j]+i)] for i in range(int(Nbins[j]))]) for j in range(len(Nbins))]

    return total_dev_by_oabin

In [ ]:
toy_ratios = [] ##toy_ratios[toy_id][oa_bin][energy_bin]
for i in range(1000):
    if (i%100 == 0):
        print(i)
    toy_ratios.append(create_toy_ratio())

weights = []
for toy in toy_ratios:
    toy_weights = []
    for oa_bin in range(len(toy)):
        toy_weights.append(1+toy[oa_bin])
    weights.append(toy_weights)

In [ ]:
for i in range(1000):
    with open(r'NewToyWeights/toy'+str(i)+'weights.txt', 'w') as fp:
        for oa_bin in weights[i]:
            # write each item on a new line
            oa_bin_str = [str(oa_bin[i]) for i in range(len(oa_bin))]
            fp.write("%s\n" % oa_bin_str)